# Chapter 2: TensorFlow 2

## 1️⃣ Chapter Overview

This chapter dives into the core of TensorFlow 2. We move away from the high-level abstract discussions of Chapter 1 into the nuts and bolts of the framework. We will explore the fundamental data structures that make up every TensorFlow program and understand how the framework executes operations under the hood.

**Key Machine Learning Concepts:**
* **The Computational Graph:** Understanding imperative (eager) vs. declarative (graph) execution.
* **Tensors:** The fundamental mathematical object in Deep Learning.
* **Variables:** Handling state and mutable parameters (weights/biases) in a network.
* **Core Operations:** Matrix multiplication, Convolution, and Pooling.

**Practical Skills:**
* Implementing a simple Neural Network (MLP) from scratch using low-level TensorFlow.
* Manipulating `tf.Tensor` and `tf.Variable` objects.
* Using the `@tf.function` decorator to compile Python code into efficient graphs.
* performing image processing tasks like edge detection using raw convolution operations.

## 2️⃣ Theoretical Explanation

### 2.1 The Big Shift: TensorFlow 1.x vs. TensorFlow 2.x

To understand TensorFlow 2, it helps to know what came before.

* **TensorFlow 1.x (Declarative/Graph-based):** You had to define a "graph" of computations first (like a blueprint), and then "run" it in a Session. It was powerful but difficult to debug because Python code didn't execute immediately.
* **TensorFlow 2.x (Imperative/Eager):** Operations are executed immediately as they are called from Python. This makes it intuitive, easy to debug, and very "Pythonic".

However, we still need the performance of graphs. TF2 bridges this gap with **AutoGraph** and the `@tf.function` decorator, which traces your Python code and converts it into a highly optimized graph automatically.

### 2.2 The Building Blocks

Every model in TensorFlow is built using three atomic elements:

1.  **`tf.Tensor` (Immutable Data):** 
    * *Definition:* A multidimensional array. Once created, its values cannot be changed.
    * *Role:* Holds input data, intermediate layer outputs, and final predictions.
    * *Analogy:* Read-only memory or constants.

2.  **`tf.Variable` (Mutable State):**
    * *Definition:* A wrapper around a tensor that allows its values to be modified (mutated).
    * *Role:* Holds the **weights** and **biases** of a neural network. These need to change during training as the model learns.

3.  **`tf.Operation` (Transformations):**
    * *Definition:* Mathematical computations that consume and produce tensors.
    * *Examples:* Matrix multiplication (`tf.matmul`), Addition (`tf.add`), Convolution (`tf.nn.conv2d`).

### 2.3 Neural Network Computations

Deep learning is essentially a composition of specific mathematical operations:

* **Matrix Multiplication:** The engine of Fully Connected (Dense) layers. It transforms input vectors by multiplying them with a weight matrix.
* **Convolution:** The engine of Computer Vision. It slides a small window (kernel) over an image to detect features like edges, textures, or shapes.
* **Pooling:** A down-sampling operation that reduces the size of data (e.g., taking the maximum value in a small window), making the model robust to small translations.

## 3️⃣ Code Reproduction: First Steps with TensorFlow 2

We will start by implementing a simple **Multilayer Perceptron (MLP)**. An MLP is a basic neural network with an input layer, hidden layers, and an output layer.

We will manually define the weights (`W`) and biases (`b`) as `tf.Variable`s and define the forward pass computation.

In [ ]:
import numpy as np
import tensorflow as tf

# 1. Define Data and Variables
# Input data: A single sample with 4 features (Shape: 1x4)
x = np.random.normal(size=[1, 4]).astype('float32')

# Initialize weights and biases randomly
init = tf.keras.initializers.RandomNormal()

# Layer 1: 4 inputs -> 3 hidden units
w1 = tf.Variable(init(shape=[4, 3]))
b1 = tf.Variable(init(shape=[1, 3]))

# Layer 2: 3 hidden units -> 2 output units
w2 = tf.Variable(init(shape=[3, 2]))
b2 = tf.Variable(init(shape=[1, 2]))

# 2. Define the Forward Pass Function
# We use @tf.function to compile this into a graph for performance
@tf.function
def forward(x, W, b, act):
    return act(tf.matmul(x, W) + b)

# 3. Execute the Model
# Hidden layer computation (Sigmoid activation)
h = forward(x, w1, b1, tf.nn.sigmoid)

# Output layer computation (Softmax activation)
y = forward(h, w2, b2, tf.nn.softmax)

print("Input x:", x)
print("Hidden h:", h.numpy())
print("Output y:", y.numpy())

### Step-by-Step Explanation

1.  **Initialization:** We create `x` as a NumPy array. In TF2, NumPy arrays are automatically converted to Tensors when passed to TF operations.
2.  **Variables:** `w1`, `b1`, `w2`, `b2` are created using `tf.Variable`. This tells TensorFlow that these are parameters we might want to update later (e.g., via gradient descent).
3.  **`@tf.function`:** This decorator is critical. When Python executes `forward`, TensorFlow traces the operations (`matmul`, `add`, `act`) and builds a graph. Subsequent calls run this optimized graph, not the Python code.
4.  **Execution:** We pass `x` through the first layer to get hidden state `h`, then pass `h` to the second layer to get output `y`.

## 4️⃣ Code Reproduction: TensorFlow Building Blocks

Let's explore `tf.Variable` and `tf.Tensor` in more detail.

In [ ]:
# --- Understanding tf.Variable ---

# Creating a variable with a specific shape and value
v1 = tf.Variable(tf.constant(2.0, shape=[4]), dtype='float32')
print("v1:", v1.numpy())

# Mutating a variable (assigning new values)
# Note: You cannot simply do v1 = new_value. You must use .assign()
v1.assign([10.0, 20.0, 30.0, 40.0])
print("v1 (after assign):", v1.numpy())

# Modifying specific slices
v_mat = tf.Variable(np.zeros(shape=[4, 3], dtype='float32'))
print("\nMatrix before:\n", v_mat.numpy())

# Assign 1.0 to the element at row 0, col 2
v_mat[0, 2].assign(1.0)
print("Matrix after single assign:\n", v_mat.numpy())

# --- Understanding tf.Tensor ---

# Tensors are immutable. You cannot do tensor[0] = 5.
a = tf.constant(4, shape=[4], dtype='float32')
b = tf.constant(2, shape=[4], dtype='float32')

# Basic Arithmetic Operations
c = a + b  # Element-wise addition
d = a * b  # Element-wise multiplication

print("\na + b:", c.numpy())
print("a * b:", d.numpy())

# Reduction Operations
large_matrix = tf.constant(np.random.normal(size=[5, 4, 3]), dtype='float32')
sum_all = tf.reduce_sum(large_matrix)
sum_axis0 = tf.reduce_sum(large_matrix, axis=0)

print("\nSum of all elements:", sum_all.numpy())
print("Sum along axis 0 shape:", sum_axis0.shape)

## 5️⃣ Code Reproduction: Neural Network Operations

### 5.1 Convolution

Convolution is the mathematical operation behind "Filters" in image processing. We will demonstrate how to perform edge detection manually using `tf.nn.convolution`.

We will:
1.  Load a sample image.
2.  Convert it to grayscale.
3.  Define a specific kernel (filter) designed to highlight edges (Laplacian filter).
4.  Apply the convolution.

*Note: The book uses 'baboon.jpg'. We will download a standard sample image to ensure this code runs immediately for you.*

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import requests
from io import BytesIO

# 1. Load an image
# We download a sample image (Lena or similar standard test image)
url = "https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png"
response = requests.get(url)
img_pil = Image.open(BytesIO(response.content))
img_pil = img_pil.resize((256, 256)) # Resize for consistency
x_rgb = np.array(img_pil).astype('float32')

# 2. Convert to Grayscale using Matrix Multiplication
# Formula: 0.3*R + 0.59*G + 0.11*B
grayscale_weights = tf.constant([[0.3], [0.59], [0.11]])
x_gray = tf.matmul(x_rgb, grayscale_weights)
# x_gray shape is (256, 256, 1)

# Display Original vs Grayscale
plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.imshow(x_rgb.astype('uint8'))
plt.title("Original")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(tf.squeeze(x_gray), cmap='gray')
plt.title("Grayscale")
plt.axis('off')

# 3. Edge Detection using Convolution
# Define an approximate Laplacian filter (detects edges)
laplacian_filter = np.array([
    [0,  1, 0],
    [1, -4, 1],
    [0,  1, 0]
], dtype='float32')

# Reshape data for TensorFlow Convolution
# Input Format: [Batch, Height, Width, Channels]
x_input = tf.reshape(x_gray, [1, 256, 256, 1])

# Filter Format: [Filter_Height, Filter_Width, In_Channels, Out_Channels]
filters = tf.reshape(laplacian_filter, [3, 3, 1, 1])

# Apply Convolution
y_conv = tf.nn.convolution(x_input, filters)

# Display Edges
plt.subplot(1, 3, 3)
plt.imshow(tf.squeeze(y_conv), cmap='gray')
plt.title("Edge Detection")
plt.axis('off')
plt.show()

### Step-by-Step Explanation

**Input:** A 3-channel RGB image.

1.  **Grayscale Conversion:** Instead of a library function, we used `tf.matmul`. We treated each pixel as a vector $[R, G, B]$ and multiplied it by the weight vector $[0.3, 0.59, 0.11]^T$. This projects the 3D color space into 1D intensity space.
2.  **Reshaping:** TensorFlow expects 4D tensors for convolution: `[Batch, Height, Width, Channels]`. Even for a single image, we add a batch dimension of size 1.
3.  **Filter Definition:** The Laplacian kernel `[[0,1,0],[1,-4,1],[0,1,0]]` sums to 0. In flat areas (constant color), the sum is $1+1+1+1-4 = 0$ (black). At edges where values change rapidly, the sum is non-zero (white/gray).
4.  **`tf.nn.convolution`:** This low-level operation slides the filter over the input. Unlike Keras layers (`Conv2D`), this does not create variables; it just performs the math using the tensors provided.

### 5.2 Pooling

Pooling reduces the size of the image, making the model computationally efficient and translation invariant. We will compare Max Pooling vs. Average Pooling.

In [ ]:
# Using the output from the previous convolution (Edge detected image)
image_input = y_conv # Shape: [1, 254, 254, 1] (size reduced slightly due to valid padding in conv)

# Max Pooling
# Window size: 2x2, Stride: 2x2
z_max = tf.nn.max_pool(image_input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

# Average Pooling
z_avg = tf.nn.avg_pool(image_input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

print(f"Original Shape: {image_input.shape}")
print(f"Pooled Shape:   {z_max.shape}")

# Visualization
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(tf.squeeze(z_max), cmap='gray')
plt.title("Max Pooling")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(tf.squeeze(z_avg), cmap='gray')
plt.title("Avg Pooling")
plt.axis('off')

plt.show()

**Analysis:**
* **Max Pooling:** Keeps the strongest feature in the window. Notice how the edges remain sharp and bright. It effectively "zooms out" while preserving dominant features.
* **Avg Pooling:** Averages the window. The result appears blurrier or "softer" because sharp edges are averaged with the black background.

## 6️⃣ Chapter Summary

* **TensorFlow 2 Philosophy:** Eager execution makes TF behave like standard Python code, while `@tf.function` provides the speed of graph execution.
* **Variables vs. Constants:** Use `tf.Variable` for model parameters (weights/biases) that need to be updated. Use `tf.constant` or `tf.Tensor` for data that remains static.
* **Math Operations:** Deep learning is built on linear algebra. `tf.matmul` is the core of most networks.
* **Computer Vision Primitives:**
    * **Convolution** detects local patterns (edges).
    * **Pooling** reduces dimensionality and adds invariance.
* **Practical Takeaway:** You can manipulate images and data using raw TensorFlow operations just like you would with NumPy, but with the added benefit of GPU acceleration and automatic differentiation (which we will cover in future chapters).